In [ ]:
from preprocessing.helpfiles import *
import random
import numpy as np
import matplotlib.pyplot as plt
import time
from matplotlib.pyplot import figure, draw, pause
import matplotlib.colors
from matplotlib.colors import LinearSegmentedColormap
from IPython.display import clear_output
import functools
import glob
import re
from PIL import Image

In [ ]:
def random_forest(width, height, vegetation_ratio=0.5, boulders=0, bouldersize=20):
    map_matrix = np.zeros((height, width))
    
    water = 0
    land = 1
    vegetation = 2
    
    for i in range(height):
        for j in range(width):
            
            # 2 tiles water padding
            if i < 2 or i > height - 3 or j < 2 or j > width - 3:
                map_matrix[i][j] = water

            # set initial vegetation
            elif random.random() < vegetation_ratio:
                map_matrix[i][j] = vegetation
            
            # set land
            else:
                map_matrix[i][j] = land
    
    for i in range(boulders):
        starting_point = [random.randint(0, height), random.randint(0, width)]
        
        for j in range(bouldersize):
            for k in range(bouldersize):
                map_matrix[(starting_point[0] + j) % height][(starting_point[1] + k) % width] = 1
        
    return map_matrix


def wind_matrix(degrees, multiplier=1):

    degrees = degrees % 360
    
    mirror = False
    if degrees > 180:
        mirror = True
        degrees -= 180
    
    n_diff = min(abs(360 - degrees), abs(360 - (degrees + 360))) / 180
    s_diff = min(abs(180 - degrees), abs(180 - (degrees + 360))) / 180
    w_diff = min(abs(270 - degrees), abs(270 - (degrees + 360))) / 180
    e_diff = min(abs(90  - degrees), abs(90  - (degrees + 360))) / 180
    
    nw_diff = min(abs(315 - degrees), abs(315 - (degrees + 360))) / 180
    sw_diff = min(abs(225 - degrees), abs(225 - (degrees + 360))) / 180
    se_diff = min(abs(135 - degrees), abs(135 - (degrees + 360))) / 180
    ne_diff = min(abs(45  - degrees), abs(45  - (degrees + 360))) / 180
    
    wind_matrix = np.zeros((3,3))
    wind_matrix[1, 1] = 1
    
    wind_matrix[0, 1] = (1 + (n_diff - .5)) ** multiplier
    wind_matrix[2, 1] = (1 + (s_diff - .5)) ** multiplier
    wind_matrix[1, 0] = (1 + (w_diff - .5)) ** multiplier
    wind_matrix[1, 2] = (1 + (e_diff - .5)) ** multiplier
    
    wind_matrix[0, 0] = (1 + (nw_diff - .5)) ** multiplier
    wind_matrix[2, 0] = (1 + (sw_diff - .5)) ** multiplier
    wind_matrix[2, 2] = (1 + (se_diff - .5)) ** multiplier
    wind_matrix[0, 2] = (1 + (ne_diff - .5)) ** multiplier
    
    if mirror:
        wind_matrix = np.flip(np.flip(wind_matrix, 0), 1)
        
    return wind_matrix / np.max(wind_matrix)

# wind_matrix(180, 5)

def rel_height_multiplier(height1, height2, tile_size):
    degree = np.arctan((height2-height1) / tile_size) / (2 *np.pi) * 360
    multiplier = np.array(((degree ** 3 * 4) / 1600000) + 0.8)
    multiplier[multiplier<0.4] = 0.4
    multiplier[multiplier>1] = 1
    return multiplier

# x_range = np.linspace(-4000,2000)
# plt.plot(x_range, rel_height_multiplier(0, x_range, 1800))
# print(rel_height_multiplier(-1000,0,1800))

In [ ]:
class ForestFires:
    def __init__(self, init_burning_ratio, wind_dir, wind_speed,
                 p_spread=0.4, p_medium=0.3, p_intense=0.2, p_ash=0.2,
                 origin_points=False):
        
        self.tile_size = 3600
        
        self.map_matrix = None
        self.height = None
        self.width = None
        
        self.sim_data = []
        self.origin_points = origin_points

        
        self.init_burning_ratio = init_burning_ratio
        
        self.p_spread = p_spread
        self.p_medium = p_medium
        self.p_intense = p_intense
        self.p_ash = p_ash
        
        
        # relative coordinates of 3x3 neighbourhood kernel
        self.neigh_dirs = np.array([[(-1,-1), 0.83], [(-1,0),1], [(-1,1), 0.83],
                                    [(0,-1),1], [(0,1),1],
                                    [(1,-1), 0.83], [(1,0),1], [(1,1), 0.83]])


        
        # init dictionary containing vegetation population

        self.new_day = True
        self.day_counter = 1
        
        self.prev_day = 0
        self.temperature_map = None
        self.precipitation_map = None
        
        self.height_map = None
        self.validation_map = None
        
        self.vegetation_map = None
        self.mask = None
        
        self.height_multiplier = None
        self.temp_multiplier = None
        self.rain_multiplier = None
        
        self.p_yellow_out = np.concatenate((np.zeros((20)), np.linspace(0,80,81) / 80))
        self.p_orange_out = np.concatenate((np.zeros((50)), np.linspace(0,50,51) / 50))
        
        self.veg_population = None
        
        self.wind = None

    def set_vegetation_population(self, map_matrix):
        veg_population = {}
        map_mat = map_matrix
        
        if not isinstance(self.origin_points, bool):
            for (y,x) in np.argwhere(map_mat == 2):
                if [y,x] in self.origin_points[0]:
                    veg_population[(x,y)] = Vegetation(0, self.temperature_map[y,x], 
                                                           self.precipitation_map[y,x], 
                                                           self.height_map[y,x])
                else:
                    veg_population[(x,y)] = Vegetation(4, self.temperature_map[y,x], 
                                                           self.precipitation_map[y,x], 
                                                           self.height_map[y,x])
        
        else:
            rand = random.random
            for (y,x) in np.argwhere(map_mat == 2):

                    # initial propability of vegetation burning
                    if rand() < self.init_burning_ratio:
                        veg_population[(x,y)] = Vegetation(0, self.temperature_map[y,x], 
                                                           self.precipitation_map[y,x], 
                                                           self.height_map[y,x])

                    else:
                        veg_population[(x,y)] = Vegetation(4, self.temperature_map[y,x], 
                                                           self.precipitation_map[y,x], 
                                                           self.height_map[y,x])
                    
        return veg_population

        

    def update(self, day):
        # create a temporary dict to safely replace values in the veg_population dict later on 
        if day != self.prev_day:
            self.temperature_map = np.flip(temperature(day, self.width, self.height),0)
            self.precipitation_map = np.flip(precipitation(day, self.width, self.height),0)
            self.prev_day = day
        # 
        temperature_map = self.temperature_map
        precipitation_map = self.precipitation_map
        height_map = self.height_map
        
        randnum = random.random
        p_spread = self.p_spread
        p_medium = self.p_medium
        p_intense = self.p_intense
        p_ash = self.p_ash
        
        # assign class attributes to local variables for improved lookup speed
        current_population = self.veg_population.copy()
        temp_population = {}
        map_matrix = self.map_matrix
        neigh_dirs = self.neigh_dirs
        wind_matrx = wind_matrix(self.wind[day][0], self.wind[day][1])
        
        for (x,y) in current_population:
            if self.new_day:
                # initiate fires according to validation data
                if [y,x] in self.origin_points[self.day_counter]:
                    temp_population[(x,y)] = Vegetation(0, self.temperature_map[y,x], 
                                                        self.precipitation_map[y,x], 
                                                        self.height_map[y,x])
            
            # if current vegetation is not burned up already
            tempr = temperature_map[y,x]
            prec = precipitation_map[y,x]
            height = height_map[y,x]
            
            if current_population[(x,y)].status == 4:
                
                # fill 3x3 neighbourhood kernel
                for ((dy, dx), distance_multiplier) in neigh_dirs:
                    y1 = y + dy
                    x1 = x + dx

                    if (x1,y1) in current_population and current_population[(x1,y1)].status == 0:
                        neighbour = current_population[(x1,y1)]
                        
                        # influence of weather conditions
#                         print(neigh.temperature)
                        t_influence = self.temp_multiplier[round(int(tempr))]
                        p_influence = self.rain_multiplier[round(int(prec))]
                        h_influence = self.height_multiplier[round(int(height))] * \
                            rel_height_multiplier(neighbour.height, height, self.tile_size)
                        w_influence = wind_matrx[dy+1,dx+1]
                        
                        weather_influence = t_influence * p_influence * h_influence * w_influence
                        if randnum() < (weather_influence * distance_multiplier):
                            temp_population[(x,y)] = Vegetation(0, tempr, prec, height)  
                            continue
                        
            elif current_population[(x,y)].status == 0:
                if randnum() < self.p_yellow_out[round(int(prec))]:
                    temp_population[(x,y)] = Vegetation(5, tempr, prec, height)
                elif randnum() < p_medium:
                    temp_population[(x,y)] = Vegetation(1, tempr, prec, height)
                    
            elif current_population[(x,y)].status == 1:
                if randnum() < self.p_orange_out[round(int(prec))]:
                    temp_population[(x,y)] = Vegetation(6, tempr, prec, height)
                elif randnum() < p_intense:
                    temp_population[(x,y)] = Vegetation(2, tempr, prec, height)

            elif current_population[(x,y)].status == 2:
                if randnum() < p_ash:
                    temp_population[(x,y)] = Vegetation(3, tempr, prec, height)
            
        for (x,y) in temp_population:
            current_population[(x,y)] = temp_population[(x,y)]
        
        self.veg_population = current_population.copy()
        
    def visualize(self, day, hour, minute, runtime, plot=True, render=False, 
                  save_as_array=False):
        current_map = self.map_matrix.copy()
#         print(np.shape(current_map))
        image = np.empty((np.shape(current_map)[0],np.shape(current_map)[1],3),dtype=object)
#         print(image)
        current_population = self.veg_population.copy()

        datetime = [str('0' + str(day) if len(str(day)) < 2 else day),
                    str('0' + str(hour) if len(str(hour)) < 2 else str(hour)),
                    str('0' + str(minute) if len(str(minute)) < 2 else str(minute))]
        
        
        
        # update burned_ratio with gradations if they are present in matrix
        for (x,y) in current_population:
            status = current_population[(x,y)].status

            if status == 0:
                current_map[y,x] = 3
                
            # start stage of fire
            elif status == 1:
                current_map[y,x] = 4
                
            # mid stage of fire
            elif status == 2:
                current_map[y,x] = 5
                
            # last stage of fire
            elif status == 3:
                current_map[y,x] = 8
                
            
            elif status == 5:
                current_map[y,x] = 6
                
            elif status == 6:
                current_map[y,x] = 7      
                
        if render:
            image = image.tolist()            
            for y in range(len(current_map)):
                for x in range(len(current_map[0])):
                    if current_map[y,x] == 0:
                        image[y][x] = [255,0,0]
                    elif current_map[y,x] == 1:
                        image[y][x] = [192,192,192]
                    elif current_map[y,x] == 2:
                        image[y][x] = [0,153,0]
                    elif current_map[y,x] == 3:
                        image[y][x] = [0,255,255]
                    elif current_map[y,x] == 4:
                        image[y][x] = [0,128,255]
                    elif current_map[y,x] == 5:
                        image[y][x] = [0,0,255]
                    elif current_map[y,x] == 6:
                        image[y][x] = [0,204,0]
                    elif current_map[y,x] == 7:
                        image[y][x] = [0,255,0]
                    elif current_map[y,x] == 8:
                        image[y][x] = [0,0,0]

            image = np.array(image) 

            font = cv2.FONT_HERSHEY_SIMPLEX
            org = (320,500)
            fontScale = .4
            fontColor = (255,255,255)
            lineType = 1

            text = f"Day: {datetime[0]} Time: {datetime[1]}:{datetime[2]}"
            

            image = cv2.putText(image, 
                                text, 
                                org, 
                                font, 
                                fontScale,
                                fontColor,
                                lineType)
            
            cv2.imwrite(f"../datasets/rendered/{datetime[0]}-{datetime[1]}-{datetime[2]}.png", image)

                    
        
        
        if plot:
            # cmap
            colorMap = ['midnightblue', 'lightgrey', 'darkgreen', 'yellow', 'orange', 'red',  'olivedrab', 'darkolivegreen', 'black']
            values, colors = list(zip(*[(i, c) for i,c in enumerate(colorMap) if i in current_map]))

            norm = plt.Normalize(min(values), max(values))
            tuples = list(zip(map(norm, values), colors))
            cmap = LinearSegmentedColormap.from_list("", tuples)
            
            # cmap2
            colors2 = ('lightgrey', 'paleturquoise', 'deepskyblue',  'mediumblue')
            values2 = (0, 20, 80, 200)
            
            norm2 = matplotlib.colors.Normalize(min(values2), max(values2))
            tuples2 = list(zip(map(norm2, values2), colors2))
            cmap2 = LinearSegmentedColormap.from_list("", tuples2)

            # plot current state of matrix
#             figure(num=None, figsize=(15, 15))



            fig = plt.figure(figsize=(16, 9))
            plt.title(f"Day = {datetime[0]}\nTime = {datetime[1]}:{datetime[2]}\nFrame runtime = {runtime}")

            ax1 = fig.add_subplot(131)
            ax1.set_axis_off()
            ax2 = fig.add_subplot(332)
            ax2.set_axis_off()
            ax3 = fig.add_subplot(335)
            ax3.set_axis_off()
            ax4 = fig.add_subplot(338)
            ax4.set_axis_off()        
            ax5 = fig.add_subplot(133)
            ax5.set_axis_off()
            
            temp_map = np.ma.masked_where(self.mask == 0, self.temperature_map)
            prec_map = np.ma.masked_where(self.mask == 0, self.precipitation_map)
            height_map = np.ma.masked_where(self.mask == 0, self.height_map)
#             val_map = np.ma.masked_where(self.mask == 0, np.flip(self.validation_map[day],0))
        
            
            ax1.imshow(current_map, cmap=cmap, norm=norm)
            ax2.imshow(temp_map, cmap='inferno')
            ax3.imshow(prec_map, interpolation='nearest', cmap=cmap2)
            ax4.imshow(height_map, cmap='YlOrBr')
            ax5.imshow(np.flip(self.validation_map[day],0), cmap='inferno')           

            plt.show()
            
        
        if save_as_array:
            np.save(f'../datasets/generated/{datetime[0]}-{datetime[1]}-{datetime[2]}', current_map)
                      
        clear_output(wait=True)
    
        
class Vegetation:
    def __init__(self, status, height, temperature, precipitation):
    
        # burning or neutral
        self.status = status
        self.temperature = temperature
        self.precipitation = precipitation
        self.height = height

In [ ]:
def forest_fire_simulation(days, iterations_per_hour, height, width, wind_dir, wind_speed, rand_forest=False,
                           vegetation_ratio=0.5, init_burning_ratio=0.05, origin_points=True):
#     map_array = None
    
    if rand_forest:
        map_array = random_forest(width, height, vegetation_ratio, boulders=0, bouldersize=20)
    else:
        data = np.flip(np.genfromtxt("../datasets/raw/veg/vegetation.grid",
                             skip_header=6, skip_footer=18), 0)
        
        map_array = construct_density_map(data, width, height, margin=0, save=False)[int(.4*height):int(.72*width),int(.8*height):width]

    if origin_points:
        origin_points = np.load('../datasets/processed/starting_points.npy', allow_pickle=True).tolist()
    forest_sim = ForestFires(map_array, init_burning_ratio, wind_dir, wind_speed, origin_points=origin_points)
        
    forest_sim.temperature_map = np.flip(temperature(0, width, height),0)[int(.4*height):int(.72*width),int(.8*height):width]
    forest_sim.precipitation_map = np.flip(precipitation(0, width, height),0)[int(.4*height):int(.72*width),int(.8*height):width]
    forest_sim.height_map = np.flip(construct_height(width, height),0)[int(.4*height):int(.72*width),int(.8*height):width]
    forest_sim.validation_map = np.load("../datasets/processed/validation_fire.npy")
    forest_sim.vegetation_map = np.flip(np.genfromtxt("../datasets/raw/veg/vegetation.grid", 
                                       skip_header=6, skip_footer=18), 0)
    
    forest_sim.height_multiplier = np.flip(np.linspace(0,2134,2135)) / 2134
    forest_sim.temp_multiplier = np.linspace(0,60,61) / 60
    forest_sim.rain_multiplier = np.flip(np.linspace(0,100,101)) / 100
    forest_sim.wind = np.load("../datasets/processed/daily_wind.npy")
    forest_sim.mask = construct_mask(forest_sim.vegetation_map, width, height)[int(.4*height):int(.72*width),int(.8*height):width]
#     print(forest_sim.mask)
    
    forest_sim.map_matrix = map_array
    forest_sim.width = width
    forest_sim.height = height
    forest_sim.veg_population = forest_sim.set_vegetation_population(forest_sim.map_matrix)
    
    

    t = time.time
    prev_runtime = 0
    for day in range(days):
        forest_sim.new_day = True
        forest_sim.day_counter += 1
        
        for hour in range(24):
#             t1 = t()
            for minute in [0,20,40]:
                t1 = t()
                forest_sim.update(day)
                if not rand_forest:
                    forest_sim.visualize(day, hour, minute, prev_runtime, plot=True, render=True, save_as_array=True)
                    t2 = t()
                    prev_runtime = t2-t1
                    
                else:
                    t1_rand = t()
                    forest_sim.visualize(day, hour, prev_runtime, plot=True, render=False)
                    t2_rand = t()
                    prev_runtime = t2_rand - t1_rand
                forest_sim.new_day = False
#             t2 = t()
#             prev_runtime = t2-t1

#             print(prev_runtime)


In [ ]:
days = 1
iterations_per_hour = 3
width = 1279
height = 1023
vegetation_ratio = 1
init_burning_ratio = 0.002
init_ratio_burned = 0.8
wind_dir = 180
wind_speed = 0
rand_forest=False

forest_fire_simulation(days, (iterations_per_hour if rand_forest else 1), height, width, wind_dir, wind_speed, rand_forest=rand_forest,
                       vegetation_ratio=vegetation_ratio, init_burning_ratio=init_burning_ratio)

In [ ]:
import ffmpeg
(
    ffmpeg
    .input('rendered/*.png', pattern_type='glob', framerate=10)
    .output('forest_fire_sim_10fps.mp4')
    .run()
)